In [1]:
import pandas as pd
from datetime import datetime
from keras.models import Sequential
from keras.layers import Dense
import tensorflow as tf
import math
import statistics
import keras


In [2]:
df_city_attr = pd.read_csv('./city_attributes.csv', sep=';')
df_humidity = pd.read_csv('./humidity_train.csv', sep=';')
df_pressure = pd.read_csv('./pressure_train.csv', sep=';')
df_temperature = pd.read_csv('./temperature_train.csv', sep=';')
df_weather_desc = pd.read_csv('./weather_description_train.csv', sep=';')
df_wind_dir = pd.read_csv('./wind_direction_train.csv', sep=';')
df_wind_speed = pd.read_csv('./wind_speed_train.csv', sep=';')


In [3]:
real_values_col_list = df_humidity.columns[1:].tolist()

for real_col in real_values_col_list:
  df_humidity.loc[pd.isna(df_humidity.loc[:, real_col]), real_col] = df_humidity[real_col].mean()
  df_pressure.loc[pd.isna(df_pressure.loc[:, real_col]), real_col] = df_pressure[real_col].mean()
  df_temperature.loc[pd.isna(df_temperature.loc[:, real_col]), real_col] = df_temperature[real_col].mean()
  df_wind_dir.loc[pd.isna(df_wind_dir.loc[:, real_col]), real_col] = df_wind_dir[real_col].mean()
  df_wind_speed.loc[pd.isna(df_wind_speed.loc[:, real_col]), real_col] = df_wind_speed[real_col].mean()


In [ ]:
df_temperature.head(5)

,datetime,Vancouver,Portland,San Francisco,Seattle,Los Angeles,San Diego,Las Vegas,Phoenix,Albuquerque,Denver,San Antonio,Dallas,Houston,Kansas City,Minneapolis,Saint Louis,Chicago,Nashville,Indianapolis,Atlanta,Detroit,Jacksonville,Charlotte,Miami,Pittsburgh,Toronto,Philadelphia,New York,Montreal,Boston,Beersheba,Tel Aviv District,Eilat,Haifa,Nahariyya,Jerusalem
0,01.10.2012 12:00,284.008592,285.017399,287.994299,284.532978,290.653231,290.076688,292.065257,295.171215,285.187253,282.505048,293.490988,292.175086,293.927417,286.534178,280.486748,286.429003,283.120827,288.363982,284.537742,289.523294,282.738699,294.149579,288.735599,298.017034,283.873509,281.792854,285.184095,285.188994,280.246221,283.653962,291.630019,294.586808,309.100000,295.310811,293.884816,292.98004
1,01.10.2012 13:00,284.630000,282.080000,289.480000,281.800000,291.870000,291.530000,293.410000,296.600000,285.120000,284.610000,289.290000,289.740000,288.270000,289.980000,286.870000,286.180000,284.010000,287.410000,283.850000,294.030000,284.030000,298.170000,288.650000,299.720000,281.000000,286.260000,285.630000,288.220000,285.830000,287.170000,307.590000,305.470000,310.580000,304.400000,304.400000,303.50000
2,01.10.2012 14:00,284.629041,282.083252,289.474993,281.797217,291.868186,291.533501,293.403141,296.608509,285.154558,284.607306,289.303649,289.762974,288.297576,289.997635,286.893636,286.185246,284.054691,287.421360,283.889394,294.035341,284.069789,298.205230,288.650172,299.732518,281.024767,286.262541,285.663208,288.247676,285.834650,287.186092,307.590000,304.310000,310.495769,304.400000,304.400000,303.50000
3,01.10.2012 15:00,284.626998,282.091866,289.460618,281.789833,291.862844,291.543355,293.392177,296.631487,285.233952,284.599918,289.338497,289.830767,288.334343,290.038151,286.951401,286.199194,284.177412,287.454637,283.941919,294.049702,284.173965,298.299595,288.650582,299.766579,281.088319,286.269518,285.756824,288.326940,285.847790,287.231672,307.391513,304.281841,310.411538,304.400000,304.400000,303.50000
4,01.10.2012 16:00,284.624955,282.100481,289.446243,281.782449,291.857503,291.553209,293.381213,296.654466,285.313345,284.592530,289.373345,289.898560,288.371111,290.078666,287.009166,286.213142,284.300133,287.487914,283.994444,294.064063,284.278140,298.393961,288.650991,299.800641,281.151870,286.276496,285.850440,288.406203,285.860929,287.277251,307.145200,304.238015,310.327308,304.400000,304.400000,303.50000


In [4]:

cities_numbers = []
count = 0
for city in df_city_attr.City:
  cities_numbers.append([city, count])
  count += 1

xd = 0

df_lists = [ [] for x in cities_numbers]

new_day = [ [df_city_attr.Latitude[i[1]], df_city_attr.Longitude[i[1]]] for i in cities_numbers]

for dt in df_humidity.datetime:
  # if(xd < 12):
  #   xd += 1
  #   continue

  datetime_object = datetime.strptime(dt, "%d.%m.%Y %H:%M")

  if(datetime_object.hour == 0):
    if(new_day):
      for i in range(len(cities_numbers)):
        df_lists[i].append(new_day[i])
    new_day = [ [df_city_attr.Latitude[x[1]], df_city_attr.Longitude[x[1]]] for x in cities_numbers]

  hum = df_humidity.loc[df_humidity.datetime == dt].values.tolist()[0]
  press = df_pressure.loc[df_pressure.datetime == dt].values.tolist()[0]
  temp = df_temperature.loc[df_temperature.datetime == dt].values.tolist()[0]
  wind_dir = df_wind_dir.loc[df_wind_dir.datetime == dt].values.tolist()[0]
  wind_speed = df_wind_speed.loc[df_wind_speed.datetime == dt].values.tolist()[0]

  cities_numbers = []
  count = 0
  for city in df_city_attr.City:
    cities_numbers.append([city, count])
    count += 1
  

  for i in range(len(cities_numbers)):
    new_row = [hum[1 + i],
                    press[1 + i],
                    temp[1 + i],
                    wind_dir[1 + i],
                    wind_speed[1 + i],
                    ]
    new_day[i].extend(new_row)


  
  # if(xd % 10 == 0):
  #   print(xd)
  xd += 1
  # if(xd >= 5000):
  #   break

# dfs = pd.DataFrame(columns = ['Latitude', 'Longitude', 'Month', 'Humidity', 'Pressure',
#                              'Temp', 'WindDir', 'WindSpeed'],
#                   data = df_list)




In [5]:
data = []

for single_city in df_lists:
  for i in range(len(single_city)):
    if i < 4:
      continue
    
    day = single_city[i]
    day1 = single_city[i - 4]
    day2 = single_city[i - 3]
    day3 = single_city[i - 2]

    Longitude = day[0]
    Latitude = day[1]

    avg_hum1 = statistics.mean([ day1[2+i] for i in range(len(day1[2:])) if i % 5 == 0])
    avg_press1 = statistics.mean([ day1[2+i] for i in range(len(day1[2:])) if i % 5 == 1])
    avg_temp1 = statistics.mean([ day1[2+i] for i in range(len(day1[2:])) if i % 5 == 2])
    avg_winddir1 = statistics.mean([ day1[2+i] for i in range(len(day1[2:])) if i % 5 == 3])
    avg_windspeed1 = statistics.mean([ day1[2+i] for i in range(len(day1[2:])) if i % 5 == 4])
    avg_hum2 = statistics.mean([ day2[2+i] for i in range(len(day2[2:])) if i % 5 == 0])
    avg_press2 = statistics.mean([ day2[2+i] for i in range(len(day2[2:])) if i % 5 == 1])
    avg_temp2 = statistics.mean([ day2[2+i] for i in range(len(day2[2:])) if i % 5 == 2])
    avg_winddir2 = statistics.mean([ day2[2+i] for i in range(len(day2[2:])) if i % 5 == 3])
    avg_windspeed2 = statistics.mean([ day2[2+i] for i in range(len(day2[2:])) if i % 5 == 4])
    avg_hum3 = statistics.mean([ day3[2+i] for i in range(len(day3[2:])) if i % 5 == 0])
    avg_press3 = statistics.mean([ day3[2+i] for i in range(len(day3[2:])) if i % 5 == 1])
    avg_temp3 = statistics.mean([ day3[2+i] for i in range(len(day3[2:])) if i % 5 == 2])
    avg_winddir3 = statistics.mean([ day3[2+i] for i in range(len(day3[2:])) if i % 5 == 3])
    avg_windspeed3 = statistics.mean([ day3[2+i] for i in range(len(day3[2:])) if i % 5 == 4])
    avg_temp5 = statistics.mean([ day[2+i] for i in range(len(day[2:])) if i % 5 == 2])
    new_row = [Latitude, Longitude, avg_hum1, avg_press1, avg_temp1, avg_winddir1, avg_windspeed1, avg_hum2, avg_press2, avg_temp2, avg_winddir2, avg_windspeed2, avg_hum3, avg_press3, avg_temp3, avg_winddir3, avg_windspeed3, avg_temp5]
    data.append(new_row)

dfs = pd.DataFrame(columns = ['Latitude', 'Longitude',
                              'Humidity1', 'Pressure1', 'Temp1', 'WindDir1', 'WindSpeed1',
                              'Humidity2', 'Pressure2', 'Temp2', 'WindDir2', 'WindSpeed2',
                              'Humidity3', 'Pressure3', 'Temp3', 'WindDir3', 'WindSpeed3',
                              'Temp5', 
                              ],
                   data = data)

In [6]:
data_class = []

for single_city in df_lists:
  for i in range(len(single_city)):
    if i < 4:
      continue
    
    day = single_city[i]
    day1 = single_city[i - 4]
    day2 = single_city[i - 3]
    day3 = single_city[i - 2]

    Longitude = day[0]
    Latitude = day[1]

    avg_hum1 = statistics.mean([ day1[2+i] for i in range(len(day1[2:])) if i % 5 == 0])
    avg_press1 = statistics.mean([ day1[2+i] for i in range(len(day1[2:])) if i % 5 == 1])
    avg_temp1 = statistics.mean([ day1[2+i] for i in range(len(day1[2:])) if i % 5 == 2])
    avg_winddir1 = statistics.mean([ day1[2+i] for i in range(len(day1[2:])) if i % 5 == 3])
    avg_windspeed1 = statistics.mean([ day1[2+i] for i in range(len(day1[2:])) if i % 5 == 4])
    avg_hum2 = statistics.mean([ day2[2+i] for i in range(len(day2[2:])) if i % 5 == 0])
    avg_press2 = statistics.mean([ day2[2+i] for i in range(len(day2[2:])) if i % 5 == 1])
    avg_temp2 = statistics.mean([ day2[2+i] for i in range(len(day2[2:])) if i % 5 == 2])
    avg_winddir2 = statistics.mean([ day2[2+i] for i in range(len(day2[2:])) if i % 5 == 3])
    avg_windspeed2 = statistics.mean([ day2[2+i] for i in range(len(day2[2:])) if i % 5 == 4])
    avg_hum3 = statistics.mean([ day3[2+i] for i in range(len(day3[2:])) if i % 5 == 0])
    avg_press3 = statistics.mean([ day3[2+i] for i in range(len(day3[2:])) if i % 5 == 1])
    avg_temp3 = statistics.mean([ day3[2+i] for i in range(len(day3[2:])) if i % 5 == 2])
    avg_winddir3 = statistics.mean([ day3[2+i] for i in range(len(day3[2:])) if i % 5 == 3])
    avg_windspeed3 = statistics.mean([ day3[2+i] for i in range(len(day3[2:])) if i % 5 == 4])
    max_windspeed5 = max([ day[2+i] for i in range(len(day[2:])) if i % 5 == 4])
    new_row = [Latitude, Longitude, avg_hum1, avg_press1, avg_temp1, avg_winddir1, avg_windspeed1, avg_hum2, avg_press2, avg_temp2, avg_winddir2, avg_windspeed2, avg_hum3, avg_press3, avg_temp3, avg_winddir3, avg_windspeed3, max_windspeed5]
    data_class.append(new_row)

dfs_class = pd.DataFrame(columns = ['Latitude', 'Longitude',
                              'Humidity1', 'Pressure1', 'Temp1', 'WindDir1', 'WindSpeed1',
                              'Humidity2', 'Pressure2', 'Temp2', 'WindDir2', 'WindSpeed2',
                              'Humidity3', 'Pressure3', 'Temp3', 'WindDir3', 'WindSpeed3',
                              'WindSpeed5', 
                              ],
                   data = data_class)

In [8]:
dfs.to_csv('./dfs_all_24h_pojutrze.csv')
dfs_class.to_csv('./dfs_class_all_24h_pojutrze.csv')

In [ ]:
dfs_class.head(10000).to_csv('./myfile.csv')

In [ ]:

dfs.head(10000).to_csv('./myfile.csv')

In [ ]:
df_shuffle = dfs.sample(frac = 1)

train_length = math.floor(df_shuffle.shape[0] * 0.8)

Xs = df_shuffle.iloc[: , :-1]
X_norm = Xs
X_norm = (Xs-Xs.mean())/Xs.std()

x_test = X_norm.iloc[train_length: , 2:]
y_test = df_shuffle.iloc[train_length: , -1:]
x_train = X_norm.iloc[:train_length , 2:]
y_train = df_shuffle.iloc[:train_length , -1:]

In [ ]:
Xs.head(10)

,Latitude,Longitude,Humidity1,Pressure1,Temp1,WindDir1,WindSpeed1,Humidity2,Pressure2,Temp2,WindDir2,WindSpeed2,Humidity3,Pressure3,Temp3,WindDir3,WindSpeed3
6276,35.216331,31.769039,38.279200,1023.083333,288.563333,279.166667,2.208333,72.100067,1015.875000,287.225833,89.375000,1.375000,81.708333,1011.875000,290.776250,128.333333,1.500000
11346,35.216331,31.769039,72.605643,1018.500000,275.255153,154.166667,4.291667,76.454971,1009.666667,275.288604,142.208333,4.625000,60.704971,1014.333333,273.652625,262.916667,7.750000
20960,35.216331,31.769039,76.125000,1027.416667,287.001708,126.958333,1.375000,86.833333,1026.750000,287.627125,79.125000,1.791667,82.875000,1024.041667,288.437813,110.958333,1.458333
4656,35.216331,31.769039,29.500000,999.916667,305.134583,209.625000,1.333333,50.375000,998.666667,302.027271,189.875000,1.750000,50.500000,999.375000,302.064708,109.625000,1.083333
7839,35.216331,31.769039,62.666667,1026.000000,290.703435,192.250000,1.375000,60.250000,1025.375000,292.477173,147.375000,1.541667,65.375000,1029.750000,291.557271,111.875000,2.833333
19294,35.216331,31.769039,80.041667,1013.833333,290.918125,154.208333,1.833333,75.958333,1014.958333,292.031875,182.166667,1.750000,77.333333,1016.083333,293.140625,147.916667,1.625000
16623,35.216331,31.769039,68.375000,1013.875000,270.862292,148.375000,2.333333,66.541667,1011.833333,261.199792,227.208333,1.958333,60.625000,1020.625000,262.313125,280.041667,1.875000
12683,35.216331,31.769039,66.125000,1030.333333,293.220615,149.416667,1.916667,71.833333,1024.125000,294.196373,176.791667,1.333333,72.541667,1019.625000,297.232337,224.625000,1.875000
8401,35.216331,31.769039,57.541667,1018.875000,299.730042,170.916667,3.416667,57.083333,1020.291667,299.966042,184.291667,3.708333,53.791667,1024.250000,301.358125,207.458333,3.833333
3937,35.216331,31.769039,12.078153,1008.750000,301.394292,124.333333,1.708333,13.536486,1003.541667,302.542083,183.541667,3.541667,22.877811,1013.041667,297.949021,237.875000,7.125000


In [ ]:
x_train.head(5)

,Humidity1,Pressure1,Temp1,WindDir1,WindSpeed1,Humidity2,Pressure2,Temp2,WindDir2,WindSpeed2,Humidity3,Pressure3,Temp3,WindDir3,WindSpeed3
17556,-0.556803,0.527779,-2.273957,1.129473,-0.461265,-0.657942,1.156541,-2.123469,-0.065014,-0.832124,0.752563,-0.021608,-1.716158,-1.537811,1.048692
16466,0.379901,0.498904,0.404414,0.164513,-0.884717,0.414816,0.431287,0.571348,-0.115842,-0.832124,0.426037,0.504770,0.601293,-1.840135,-0.673151
21452,1.782746,0.043321,1.356596,0.913611,0.544434,0.901703,-0.380613,1.351706,0.930507,0.359603,1.221134,-0.149992,1.330899,1.736295,0.916243
10452,-0.038784,0.357738,-1.792459,-0.522161,0.650297,0.345260,0.944742,-2.595598,1.543945,1.339467,-0.621729,1.223724,-2.824410,1.117612,0.465914
11912,0.068478,0.008029,-0.662518,0.569983,-0.011347,-1.268780,0.390759,-1.196978,1.272864,0.862776,-1.079099,0.883504,-1.134093,-1.037251,-0.805600


In [ ]:
dim = x_test.shape[1]
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=0.1,
    decay_steps=100,
    decay_rate=0.9)
adam = tf.keras.optimizers.Adam(
    learning_rate=lr_schedule)
model = Sequential()
model.add(Dense(12, input_dim=dim, activation='relu'))
model.add(Dense(10, activation='relu'))
model.add(Dense(10, activation='relu'))
model.add(Dense(1, activation='relu'))
model.compile(loss='mse', optimizer=adam)

In [ ]:
model.fit(x=x_train, y=y_train, epochs=50)

In [ ]:
predicted = model.predict(x_test)
counter = 0
for i in range(len(predicted)):
  if abs(abs(predicted[i][0] -  y_test.iloc[i].values[0])) <= 2:
    counter+=1

print(counter)
print(counter/(len(predicted)))

3346
0.748211091234347


In [ ]:
df_class_shuffle = dfs_class.sample(frac = 1)

train_class_length = math.floor(df_class_shuffle.shape[0] * 0.8)

Xs = df_class_shuffle.iloc[: , :-1]
X_norm = Xs
X_norm = (Xs-Xs.mean())/Xs.std()

x_test = X_norm.iloc[train_length: , 2:]
y_test = df_class_shuffle.iloc[train_length: , -1:]
x_train = X_norm.iloc[:train_length , 2:]
y_train = df_class_shuffle.iloc[:train_length , -1:]

#zamiana y na {0,1}
y_train.loc[y_train['WindSpeed5'] < 6.0, 'WindSpeed5'] = 0
y_train.loc[y_train['WindSpeed5'] >= 6.0, 'WindSpeed5'] = 1
y_test.loc[y_test['WindSpeed5'] < 6.0, 'WindSpeed5'] = 0
y_test.loc[y_test['WindSpeed5'] >= 6.0, 'WindSpeed5'] = 1

In [ ]:
print(y_train.head(20))

       WindSpeed5
21062         0.0
16862         1.0
5837          0.0
10758         1.0
21426         0.0
4721          0.0
16021         1.0
19684         1.0
6965          1.0
3291          1.0
2032          0.0
12682         1.0
8351          1.0
8255          0.0
58            1.0
5247          1.0
12111         0.0
8977          1.0
11461         0.0
3561          0.0


In [ ]:
dim = x_test.shape[1]
model = Sequential()
model.add(Dense(12, input_dim=dim, activation='sigmoid'))
model.add(Dense(10, activation='sigmoid'))
model.add(Dense(10, activation='sigmoid'))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
model.fit(x=x_train, y=y_train, epochs=50)

Epoch 1/50
559/559 [==============================] - 1s 2ms/step - loss: 0.5954 - accuracy: 0.6782
Epoch 2/50
559/559 [==============================] - 1s 2ms/step - loss: 0.5951 - accuracy: 0.6773
Epoch 3/50
559/559 [==============================] - 1s 2ms/step - loss: 0.5944 - accuracy: 0.6795
Epoch 4/50
559/559 [==============================] - 1s 2ms/step - loss: 0.5944 - accuracy: 0.6789
Epoch 5/50
559/559 [==============================] - 1s 2ms/step - loss: 0.5940 - accuracy: 0.6782
Epoch 6/50
559/559 [==============================] - 1s 2ms/step - loss: 0.5937 - accuracy: 0.6808
Epoch 7/50
559/559 [==============================] - 1s 2ms/step - loss: 0.5935 - accuracy: 0.6795
Epoch 8/50
559/559 [==============================] - 1s 2ms/step - loss: 0.5929 - accuracy: 0.6790
Epoch 9/50
559/559 [==============================] - 1s 2ms/step - loss: 0.5927 - accuracy: 0.6814
Epoch 10/50
559/559 [==============================] - 1s 2ms/step - loss: 0.5925 - accuracy: 0.6796

KeyboardInterrupt: ignored